In [ ]:
import pandas as pd
from psmpy import PsmPy
from sklearn.neighbors import NearestNeighbors

In [ ]:
# file containing normalised covariates (with UID ie. primary key)
df = pd.read_csv("*.csv")
norm = df.dropna()
metadata = [] # depends on the data, remove everything except covariates

### Note that index in the csv file has the name idx here (not id or index). Change that correspondingly.

In [ ]:
psm = PsmPy(norm, treatment='Treatment', indx='idx', exclude = ['lon','lat'] + metadata)
psm.logistic_ps(balance = True)

In [ ]:
p_scores = psm.predicted_data.copy()
p_scores.set_index('idx',inplace=True)
p_scores.sort_index(inplace=True)
p_scores = p_scores[['propensity_score','propensity_logit']]
p_scores

In [ ]:
dataset_psm = norm.join(p_scores)
dataset_psm

In [ ]:
def get_matching_pairs(treated_df, non_treated_df):
    treated_x = treated_df.values
    non_treated_x = non_treated_df.values
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='brute').fit(non_treated_x)
    distances, indices = nbrs.kneighbors(treated_x)
    indices = indices.reshape(indices.shape[0])
    matched = non_treated_df.iloc[indices]
    return matched

In [ ]:
df = dataset_psm.copy()
df.reset_index(inplace=True)
df

In [ ]:
all_matchings = pd.DataFrame()
treated_df = df[(df['Treatment']==1)][['idx','propensity_logit']].set_index('idx')
non_treated_df = df[(df['Treatment']==0)][['idx','propensity_logit']].set_index('idx')
if len(treated_df)>0 and len(non_treated_df)>0:
  matched_df = get_matching_pairs(treated_df, non_treated_df)
  all_matching = pd.DataFrame({'idx': treated_df.index, 'matched_ID': matched_df.index})

#### change this accordingly as per the data you have. Try to write in the sae order as given in the csv file.

In [ ]:
matching = all_matchings
fields = ['idx'] + metadata + ['lat', 'lon', 'dist_closest_lin', 'dist_closest_river', 'dist_closest_road', 'dist_closest_upstream_forest', 'elevation',
 'slope', 'flow_accumulation', 'proximity_water',  'drainage_density', 
#  'cropping_intensity_2016-2017',  
 'HSG',
 'CEC',
 'PH',
 'OC']
print(len(fields))

In [ ]:

all_fields = fields + [a+'_matched' for a in fields]
asset_cvts = pd.merge(matching, norm,  how='inner', left_on=['idx'], right_on = ['idx'])
asset_cfs_cvts= pd.merge(asset_cvts, norm,  how='inner', left_on=['matched_ID'], right_on = ['idx'])
asset_cfs_cvts = asset_cfs_cvts.drop(columns=['Treatment_x','Treatment_y','matched_ID'])
asset_cfs_cvts.columns = all_fields
matchings_df = asset_cfs_cvts
matchings_df.to_csv("./matched.csv", index=False)
asset_cfs_cvts

## integraring the NDVI values after matching and DiD computation by Harshit